# Apply PCA

This notebook exploits scikit learning tools.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.decomposition import PCA
from os.path import join

Let's define some global parameters for the analysis.
- <code>n_components</code> is the dimension of the PCA reduction
- <code>n_clusters</code> is the number of clusters for the k-means

In [ ]:
n_components = 3
n_clusters = 10

Load embeddings data. Although the file extension is JSON the format is actually not. Neither numpy nor pandas manage to load the data, hence we simply read the text file and convert to numpy array its values.

In [ ]:
modelpath = '../data/latent_space/SimCLR' 
filename = join(modelpath, 'embeddings.json')
with open(filename, 'r') as f:
    text = f.read()

embeddings = np.array(eval(text))
embeddings

Check data shape.

In [ ]:
np.shape(embeddings)

Instantiate PCA. 

If <code>n_components</code> parameter is not set then it will keep all of them by default. If <code>n_components == 'mle'</code> and <code>svd_solver == 'full'</code>, Minka’s MLE is used to guess the dimension. Use of <code>n_components == 'mle'</code> will interpret <code>svd_solver == 'auto'</code> as <code>svd_solver == 'full'</code>. Requires <code>n_samples >= n_features</code>. 

We want 3D reduction hence we use <code>n_components=3</code>.

In [ ]:
pca = PCA(n_components=n_components)

Fit the model with data.

In [ ]:
pca.fit(embeddings)

### Explore the PCA results

Check the amount of variance explained by each of the selected components. The variance estimation uses <code>n_samples - 1</code> degrees of freedom. It is equal to <code>n_components</code> largest eigenvalues of the covariance matrix.

In [ ]:
pca.explained_variance_

Check principal axes in feature space, representing the directions of maximum variance in the data. Equivalently, the right singular vectors of the centered input data, parallel to its eigenvectors. The components are sorted by <code>explained_variance_</code>.

In [ ]:
pca.components_

Check number of samples and features.

In [ ]:
pca.n_samples_, pca.n_features_

Check singular values corresponding to each of the selected components. The singular values are equal to the 2-norms of the <code>n_components</code> variables in the lower-dimensional space.

In [ ]:
pca.singular_values_

Check percentage of variance explained by each of the selected components.

In [ ]:
pca.explained_variance_ratio_

Print labels.

In [ ]:
labels = pca.get_feature_names_out(input_features=None)
labels

Fit the model with data and apply the dimensionality reduction.

In [ ]:
reduced = pca.fit_transform(embeddings)
np.shape(reduced)

In [ ]:
%matplotlib notebook

fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")

ax.scatter(
    reduced[:, 0],
    reduced[:, 1],
    reduced[:, 2],
    c='b',
    cmap=plt.cm.Set1,
    edgecolor="k",
    s=40,
)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

# Apply k-means

Apply k-means clustering algorithm on PCA reduced data.

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(reduced)

Extract labels.

In [ ]:
labels = kmeans.labels_
labels

Predict cluster association.

In [ ]:
predicted = kmeans.predict(reduced)
predicted

Compute cluster centers.

In [ ]:
centers = kmeans.cluster_centers_
centers

Create associated color array.

In [ ]:
colors = np.arange(0, n_clusters, 1)
colors

Plot PCA reduced data and cluster centers, using cluster association color-coding.

In [ ]:
fig = plt.figure(1, figsize=(8, 6))
ax = fig.add_subplot(111, projection="3d")

ax.scatter(
    reduced[:, 0],
    reduced[:, 1],
    reduced[:, 2],
    c=predicted.astype(float),
    cmap=plt.cm.Set1,
    edgecolor="k",
    s=40,
)

ax.scatter(
    centers[:, 0],
    centers[:, 1],
    centers[:, 2],
    marker='*',
    c=colors,
    cmap=plt.cm.Set1,
    edgecolor="k",
    s=200,
)

ax.set_xlabel("x")
ax.set_ylabel("y")
ax.set_zlabel("z")

plt.show()

## Visualise images from each cluster

Let's display a few images associated with each cluster in order to understand which features were learned by the model, and label the clusters with human-readable naming. 

**NdA:** pandas complains about mismatching colum lenghts due to "index" and "data" being empty. I have removed manually these columns from the file.

In [ ]:
filename = '../data/latent_space/SimCLR/labels.json'

imgname = pd.read_json(filename)
imgname.rename(columns={'columns': 'image'}, inplace=True)
imgname.head()

Assuming that the embeddings and labels files are ordinately written, meaning that each element index correspond between the two files, we can visualise a sample of images associated with each cluster. First let's convert the predicted array into a pandas DF.

In [ ]:
clustered = pd.DataFrame(predicted, columns=['cluster'])
clustered.head()

Now we can join row-wise the two database, cluster association and image name.

In [ ]:
associations = pd.concat([clustered, imgname], axis=1)
associations.head()

Now we can display the first **ground truth** images in each cluster.

In [ ]:
fig, axes = plt.subplots(ncols=5, nrows=n_clusters, figsize=(20, 4*n_clusters))

for c in colors:
    indexes = associations[associations['cluster']==c].head().index
    axes[c][0].set_ylabel(f'cluster #{c}')
    for i, idx in enumerate(indexes):
        png = join(modelpath, 'images', associations[associations['cluster']==c]['image'][idx])
        img = mpimg.imread(png)
        axes[c][i].imshow(img)
        axes[colors[-1]][i].set_xlabel(f'ground truth image #{i}')
    
plt.show()

And now we can do the same for the generated features map.

In [ ]:
fig, axes = plt.subplots(ncols=5, nrows=n_clusters, figsize=(20, 4*n_clusters))

for c in colors:
    indexes = associations[associations['cluster']==c].head().index
    axes[c][0].set_ylabel(f'cluster #{c}')
    for i, idx in enumerate(indexes):
        png = join(modelpath, 'generated', associations[associations['cluster']==c]['image'][idx])
        img = mpimg.imread(png)
        axes[c][i].imshow(img)
        axes[colors[-1]][i].set_xlabel(f'generated feature map #{i}')
    
plt.show()